In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SparkSession
import re

In [ ]:
def rdd_to_hdfs(rdd):
  if not rdd.isEmpty():
    spark=SparkSession.builder.appName("App").getOrCreate()
    #l=[]

    lst=rdd.collect()
    hdfs_path="hdfs://localhost:54310/user/ubuntu/datasets"

    #Each line is stored separately due to possible inconsistencies on number of fields
    for s in lst:
      #l.append(str(s).split(","))

      # converting to list because It needs to be iterable
      l=[str(s).split("/t")]
      # l=[]
      # modified_s=re.sub(r"''+", "' '", str(s))
      # for field in re.findall(r"'.+?'|\w+", modified_s):
      #   l.append(field)

      #print(s)
      #print(l)
      #print(len(l))

      schema=""
      for i in range(len(l)):
        schema=schema+(chr(ord('`')+(i+1)))+" string, "

      schema=schema[0:len(schema)-2]
      #print(schema)

      try:
        df=spark.createDataFrame(l, schema=schema)
        df.write.csv(hdfs_path, mode="append", header=False)
      except Exception as e:
        pass

    # schema=""
    # for i in range(len(l[0])):
    #   schema=schema+(chr(ord('`')+(i+1)))+" string, "
    #
    # schema=schema[0:len(schema)-2]
    #
    # df=spark.createDataFrame(l, schema=schema)
    # #df.show()
    #
    # hdfs_path="hdfs://localhost:54310/user/ubuntu/datasets"
    # df.write.csv(hdfs_path, mode="append", header=False)

In [ ]:
# Create a local StreamingContext with two working thread and batch interval of 1 second
sc = SparkContext("local[2]")
sc.setLogLevel("ERROR")
ssc = StreamingContext(sc, 1)

# Create a DStream that will connect to hostname:port, like localhost:9999
lines = ssc.socketTextStream("localhost", 9999)

batch=lines.map(lambda line: line.split("\n"))

batch.foreachRDD(rdd_to_hdfs)

ssc.start()             # Start the computation
ssc.awaitTermination()  # Wait for the computation to terminate

In [ ]:
sc.stop()